<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
# librerias
import pickle
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

# 1.Clasificador
### Cargar modelo

In [187]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
replace clasificador/variables/variables.data-00000-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace clasificador/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace clasificador/variables/variables.data-00001-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace clasificador/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [188]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento
Cargamos el label_tokenizer, el encodeer, los trigramas y bigramas.

In [189]:
#label tokenizer
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

In [190]:
#encoder
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/encoder.pickle', 'rb') as f:
  encoder = pickle.load(f)

In [191]:
#trigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [192]:
#bigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

### Función clasificadora

In [193]:
# funcion de frase tokenizada a palabras
reverse_word_map = dict(map(reversed, label_tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Busca palabras en el diccionario
    label = [reverse_word_map.get(i) for i in list_of_indices]
    return(label)

In [231]:
def clasifica (sentence):
  sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
  sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
  # preprocesamiento trigramas
  vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
  X3 = vectorizer3.fit_transform([sentence])
  # preprocesamiento bigramas
  vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
  X2 = vectorizer2.fit_transform([sentence])
  # concatenamos
  sequences_bt = np.concatenate((X3.toarray(), X2.toarray()), axis=1)
  # predicción
  pred = np.argmax(modelo_clasificador.predict(sequences_bt), axis=-1) #prediccion del modelo
##pred = encoder.inverse_transform(pred) #de label encoder a tokenizer#################### REVISAR ###################
  pred = sequence_to_text(pred) #de tokenizer a label
  
  return (pred)

In [232]:
#ejemplo
sentence = "Would this be a good time to be honest?"
clasifica(sentence)

['marge_simpson']

In [233]:
clasifica("Manzana")

['<OOV>']

# 2.Generador
### Cargar modelo

In [196]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip
replace generador/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace generador/variables/variables.data-00000-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace generador/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace generador/variables/variables.data-00001-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [197]:
modelo_generador = tf.keras.models.load_model('generador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento

In [198]:
# char a id
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/charxid.pickle', 'rb') as f:
  charxid = pickle.load(f)

In [199]:
#id a char
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/idxchar.pickle', 'rb') as f:
  idxchar = pickle.load(f)

### Función generadora

In [200]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(start_string, num_generate):

  #convertimos el texto inicial en numeros (vectorizacion)
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension

  #guardamos el texto predicho
  text_generated = []

  temperature = 1.0

  #tamaño del bache== 1
  modelo_generador.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = modelo_generador(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

In [201]:
#ejemplo
sentence = "Homer:Would this be a good time to be honest?"
generate_text(sentence, 1000)

"Homer:Would this be a good time to be honest?\r\nTake: Where remost friends made trying the first test are a hind? I have the friends of chill!\r\nLisa Simpson: Dad! Let was now not this could be a minute. (STING OFF POME WALEN'T CHINDS) Okay, okay.\r\nBart Simpson: I saw it hat you anyone employaro, Krusty twishing. You excitize me finally. Of no as the room feels let you, sir. The lives are anything works to show the losses, Mall... uh, .'ve got that real.\r\nMarge Simpson: I just can teaching you're prying to show everybody. Mawead of shows to Mr. Party actually, not you're my new lot of more.\r\nColocial: Thank you for nothing.\r\nFat Host: Your business.\r\nLisa Simpson: Bart Simpson?\r\nBart Simpson: Othere, I am working.\r\nLisa Simpson: I like to me!\r\nCarl Carlson: You can't. (CHUCKLES OF TRYT) Okay, Bart. Lough, Mr. Simpson, huh.\r\nC. Montgomery Burns: I've to have again condivited an orination. But the resired of the gless.\r\nTruck Manager: I'm ruining. I've can't comple

#Funciones para unir redes

In [202]:
import random
import re 

In [203]:
# frases de respuesta si no logra clasificar
frases_respuesta = ["Mmm, donuts.",
                      "Whatever, I'll be at Moe's.",
                      "Eat my shorts!",
                      "If anyone wants me, I'll be in my room.",
                      "Bye!",
                      "To start press any key. Where's the ANY key?",
                      "The lesson is, never try",
                      "I'm normally not a praying man, but if you're up there, please save me, Superman",
                      "You'll have to speak up I'm wearing a towel",
                      "I am so smart, S-M-R-T",
                    
                    "You're making a scene",
                      "Can I have some money now?"]

In [204]:
def union(lst1, lst2): 
    if lst1 != "<OOV>":
        final_list = lst1 + ": " + lst2 + "\n"
        #final_list = final_list.split('\n')[0]
        #final_list = generate_text(Transicion, 1000)
        #final_list = output_ultimate(final_list)
        return final_list
    else:
        a = random.randint(0, len(frases_respuesta)) #elige una frase aleatoriamente
        return frases_respuesta[a]

In [205]:
def output_ultimate(line):
  if line.split('\n')[1] == ' ':
    line2=line.split('\n')[2]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)
  else:
    line2=line.split('\n')[1]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)

In [206]:
#def etiqueta(display_one):
#  if display_one != '<OOV>':
#    display_one = display_one[:display_one.index('_')].capitalize()
#  return (display_one)

# Pre chatbot

In [221]:
def prechatbot(Sentence):
  Pre_Etiqueta=clasifica(Sentence)
  display_one = ''.join(Pre_Etiqueta)
  Etiqueta = display_one[:display_one.index('_')].capitalize()
  Transicion = union(Etiqueta, Sentence).split('\n')[0]
  Linea = generate_text(Transicion, 1000)
  output_ultimate(Linea)

In [208]:
#def prechatbot(Sentence):
#  Pre_Etiqueta=clasifica(Sentence)
#  display_one = ''.join(Pre_Etiqueta)
#  Etiqueta = etiqueta(display_one)
#  Transicion = union(Etiqueta, Sentence)
#  return (Transicion)

In [222]:
#ejemplo
Usuario = "Would this be a good time to be honest?"
prechatbot(Usuario)

 (QUICKLY) You brought the some gun for out!


In [223]:
#ejemplo
Usuario = "Hello, how are you? Are you fine?"
prechatbot(Usuario)

 I fisned the girls from this m.


In [224]:
#ejemplo
Usuario = "Manzana"
#prechatbot(Usuario)
clasifica(Usuario)

['<OOV>']

In [212]:
#ejemplo
Usuario = "Hello"
prechatbot(Usuario)

ValueError: ignored